# Visualizaciones y Análisis Visual - Superstore Dataset

En este notebook creamos visualizaciones avanzadas para:
- Analizar patrones temporales de ventas
- Comparar rendimiento por regiones y categorías
- Identificar tendencias estacionales
- Crear dashboards interactivos
- Generar insights accionables para el negocio

## 1. Configuración inicial y carga de datos

Importamos librerías y cargamos los datos procesados del notebook anterior.

In [ ]:
# Importamos las librerías necesarias para visualización
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from datetime import datetime
import warnings

# Importamos nuestras funciones auxiliares
import sys
sys.path.append('../scripts')
from utils import *

# Configuraciones
warnings.filterwarnings('ignore')
configurar_estilo_graficos()

# Configuramos Plotly para mostrar gráficos en el notebook
import plotly.io as pio
pio.renderers.default = "notebook"

print("📊 Librerías de visualización cargadas correctamente")

In [ ]:
# Cargamos los datos procesados
try:
    df = pd.read_csv('../data/superstore_procesado.csv')
    # Convertimos fechas de nuevo a datetime
    df['fecha_pedido'] = pd.to_datetime(df['fecha_pedido'])
    df['fecha_envio'] = pd.to_datetime(df['fecha_envio'])
    print(f"✅ Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")
except FileNotFoundError:
    print("⚠️ Archivo procesado no encontrado. Cargando datos originales...")
    df = cargar_datos('../data/Sample - Superstore.csv')
    df = procesar_fechas(df)
    print(f"✅ Datos originales cargados y procesados: {df.shape[0]} filas, {df.shape[1]} columnas")

## 2. Dashboard de Métricas Principales

Creamos un dashboard con las métricas más importantes del negocio.

In [5]:
# Calculamos métricas principales
total_ventas = df['ventas'].sum()
total_ganancia = df['ganancia'].sum()
margen_promedio = (total_ganancia / total_ventas) * 100
total_pedidos = df['id_pedido'].nunique()
total_clientes = df['id_cliente'].nunique()
ticket_promedio = total_ventas / total_pedidos

# Creamos un dashboard de métricas con mejor espaciado y tamaño de fuente
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=('Ventas Totales', 'Ganancia Total', 'Margen Promedio', 
                   'Total Pedidos', 'Total Clientes', 'Ticket Promedio'),
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}],
           [{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}]],
    vertical_spacing=0.18,  # Menor separación vertical para evitar amontonamiento
    horizontal_spacing=0.08 # Más espacio horizontal entre columnas
 )

# Añadimos indicadores con mayor tamaño de número y título más compacto
fig.add_trace(go.Indicator(
    mode="number",
    value=total_ventas,
    title={"text": "<b>Ventas Totales ($)</b>", "font": {"size": 16}},
    number={'valueformat': ',.0f', 'font': {'size': 32}}
), row=1, col=1)

fig.add_trace(go.Indicator(
    mode="number",
    value=total_ganancia,
    title={"text": "<b>Ganancia Total ($)</b>", "font": {"size": 16}},
    number={'valueformat': ',.0f', 'font': {'size': 32}}
), row=1, col=2)

fig.add_trace(go.Indicator(
    mode="number",
    value=margen_promedio,
    title={"text": "<b>Margen Promedio (%)</b>", "font": {"size": 16}},
    number={'valueformat': '.1f', 'font': {'size': 32}}
), row=1, col=3)

fig.add_trace(go.Indicator(
    mode="number",
    value=total_pedidos,
    title={"text": "<b>Total Pedidos</b>", "font": {"size": 16}},
    number={'valueformat': ',d', 'font': {'size': 32}}
), row=2, col=1)

fig.add_trace(go.Indicator(
    mode="number",
    value=total_clientes,
    title={"text": "<b>Total Clientes</b>", "font": {"size": 16}},
    number={'valueformat': ',d', 'font': {'size': 32}}
), row=2, col=2)

fig.add_trace(go.Indicator(
    mode="number",
    value=ticket_promedio,
    title={"text": "<b>Ticket Promedio ($)</b>", "font": {"size": 16}},
    number={'valueformat': ',.0f', 'font': {'size': 32}}
), row=2, col=3)

fig.update_layout(
    title=" Dashboard de Métricas Principales del Negocio",
    height=600,  # Más alto para evitar solapamiento
    width=1100,  # Más ancho para mejor distribución
    font=dict(size=15),
    margin=dict(t=80, l=30, r=30, b=30),
    plot_bgcolor='white',
    paper_bgcolor='white',
    title_font_size=22,
    title_x=0.5,
    title_y=0.97,
    title_font_family='Arial'
 )

fig.show()

## 3. Análisis Temporal Detallado

Analizamos la evolución temporal de las ventas con diferentes granularidades.

In [ ]:
# Evolución temporal de ventas y ganancias
ventas_temporales = df.groupby(df['fecha_pedido'].dt.to_period('M')).agg({
    'ventas': 'sum',
    'ganancia': 'sum',
    'id_pedido': 'nunique'
}).reset_index()

ventas_temporales['fecha_pedido'] = ventas_temporales['fecha_pedido'].dt.to_timestamp()
ventas_temporales['margen'] = (ventas_temporales['ganancia'] / ventas_temporales['ventas']) * 100

# Creamos gráfico con doble eje Y
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Evolución de Ventas y Ganancias Mensuales', 'Evolución del Margen de Ganancia'),
    specs=[[{"secondary_y": True}], [{"secondary_y": False}]],
    vertical_spacing=0.15
)

# Gráfico de ventas
fig.add_trace(
    go.Scatter(x=ventas_temporales['fecha_pedido'], y=ventas_temporales['ventas'],
               mode='lines+markers', name='Ventas ($)', line=dict(color='blue', width=3)),
    secondary_y=False, row=1, col=1
)

# Gráfico de ganancias
fig.add_trace(
    go.Scatter(x=ventas_temporales['fecha_pedido'], y=ventas_temporales['ganancia'],
               mode='lines+markers', name='Ganancia ($)', line=dict(color='green', width=3)),
    secondary_y=True, row=1, col=1
)

# Gráfico de margen
fig.add_trace(
    go.Scatter(x=ventas_temporales['fecha_pedido'], y=ventas_temporales['margen'],
               mode='lines+markers', name='Margen (%)', line=dict(color='red', width=3),
               fill='tonexty'),
    row=2, col=1
)

# Configuramos ejes
fig.update_xaxes(title_text="Fecha", row=2, col=1)
fig.update_yaxes(title_text="Ventas ($)", secondary_y=False, row=1, col=1)
fig.update_yaxes(title_text="Ganancia ($)", secondary_y=True, row=1, col=1)
fig.update_yaxes(title_text="Margen (%)", row=2, col=1)

fig.update_layout(
    title="📈 Análisis Temporal Detallado",
    height=700,
    hovermode='x unified'
)

fig.show()

## 4. Análisis Estacional y Patrones Semanales

Identificamos patrones estacionales y tendencias semanales en las ventas.

In [6]:
# Análisis por día de la semana
df['dia_semana_num'] = df['fecha_pedido'].dt.dayofweek
df['nombre_dia'] = df['fecha_pedido'].dt.day_name()

ventas_dia_semana = df.groupby(['nombre_dia', 'dia_semana_num'])['ventas'].sum().reset_index()
ventas_dia_semana = ventas_dia_semana.sort_values('dia_semana_num')

# Análisis por mes
df['nombre_mes'] = df['fecha_pedido'].dt.month_name()
df['mes_num'] = df['fecha_pedido'].dt.month
ventas_mes = df.groupby(['nombre_mes', 'mes_num'])['ventas'].sum().reset_index()
ventas_mes = ventas_mes.sort_values('mes_num')

# Creamos subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Ventas por Día de la Semana', 'Ventas por Mes', 
                   'Heatmap: Ventas por Mes y Año', 'Análisis de Trimestres'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'heatmap', 'colspan': 2}, None]],
    vertical_spacing=0.15
)

# Gráfico por día de la semana
fig.add_trace(
    go.Bar(x=ventas_dia_semana['nombre_dia'], y=ventas_dia_semana['ventas'],
           marker_color='skyblue', name='Ventas por Día'),
    row=1, col=1
)

# Gráfico por mes
fig.add_trace(
    go.Bar(x=ventas_mes['nombre_mes'], y=ventas_mes['ventas'],
           marker_color='lightcoral', name='Ventas por Mes'),
    row=1, col=2
)

# Heatmap mes-año
pivot_mes_año = df.pivot_table(values='ventas', index='mes_num', columns='año', aggfunc='sum')
fig.add_trace(
    go.Heatmap(
        z=pivot_mes_año.values,
        x=pivot_mes_año.columns,
        y=[f"Mes {i}" for i in pivot_mes_año.index],
        colorscale='Viridis',
        name='Ventas Mes-Año'
    ),
    row=2, col=1
)

fig.update_layout(
    title=" Análisis de Patrones Estacionales",
    height=800,
    showlegend=False
)

fig.show()

## 5. Análisis Geográfico

Exploramos el rendimiento por regiones, estados y ciudades.

In [7]:
# Análisis por región
ventas_region = df.groupby('region').agg({
    'ventas': 'sum',
    'ganancia': 'sum',
    'cantidad': 'sum',
    'id_cliente': 'nunique',
    'id_pedido': 'nunique'
}).round(2)

ventas_region['margen'] = (ventas_region['ganancia'] / ventas_region['ventas'] * 100).round(2)
ventas_region['ticket_promedio'] = (ventas_region['ventas'] / ventas_region['id_pedido']).round(2)

# Gráficos regionales
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Ventas por Región', 'Margen por Región', 
                   'Top 10 Estados por Ventas', 'Top 10 Ciudades por Ventas'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}]]
)

# Ventas por región
fig.add_trace(
    go.Bar(x=ventas_region.index, y=ventas_region['ventas'],
           marker_color='steelblue', name='Ventas'),
    row=1, col=1
)

# Margen por región
fig.add_trace(
    go.Bar(x=ventas_region.index, y=ventas_region['margen'],
           marker_color='forestgreen', name='Margen %'),
    row=1, col=2
)

# Top estados
top_estados = df.groupby('estado')['ventas'].sum().sort_values(ascending=False).head(10)
fig.add_trace(
    go.Bar(y=top_estados.index, x=top_estados.values, orientation='h',
           marker_color='coral', name='Estados'),
    row=2, col=1
)

# Top ciudades
top_ciudades = df.groupby('ciudad')['ventas'].sum().sort_values(ascending=False).head(10)
fig.add_trace(
    go.Bar(y=top_ciudades.index, x=top_ciudades.values, orientation='h',
           marker_color='gold', name='Ciudades'),
    row=2, col=2
)

fig.update_layout(
    title=" Análisis Geográfico de Ventas",
    height=800,
    showlegend=False
)

fig.show()

print(" Resumen por región:")
display(ventas_region)

 Resumen por región:


,ventas,ganancia,cantidad,id_cliente,id_pedido,margen,ticket_promedio
region,,,,,,,
Central,501239.89,39706.36,8780,629,1175,7.92,426.59
East,678781.24,91522.78,10618,674,1401,13.48,484.50
South,391721.90,46749.43,6209,512,822,11.93,476.55
West,725457.82,108418.45,12266,686,1611,14.94,450.32


## 6. Análisis de Productos y Categorías

Analizamos el rendimiento de productos, categorías y subcategorías.

In [8]:
# Análisis detallado por categoría y subcategoría
analisis_productos = df.groupby(['categoria', 'subcategoria']).agg({
    'ventas': 'sum',
    'ganancia': 'sum',
    'cantidad': 'sum',
    'descuento': 'mean'
}).round(2)

analisis_productos['margen'] = (analisis_productos['ganancia'] / analisis_productos['ventas'] * 100).round(2)

# Top subcategorías por ventas
top_subcategorias = analisis_productos.sort_values('ventas', ascending=False).head(15)

# Treemap de categorías
categoria_data = df.groupby('categoria').agg({
    'ventas': 'sum',
    'ganancia': 'sum'
}).reset_index()

fig_treemap = px.treemap(
    categoria_data,
    path=['categoria'],
    values='ventas',
    color='ganancia',
    title=' Distribución de Ventas por Categoría (Treemap)',
    color_continuous_scale='RdYlBu_r',
    labels={'ganancia': 'Ganancia ($)', 'ventas': 'Ventas ($)'}
)

fig_treemap.update_layout(height=400)
fig_treemap.show()

# Gráfico de dispersión: Ventas vs Margen por subcategoría
fig_scatter = px.scatter(
    analisis_productos.reset_index(),
    x='ventas',
    y='margen',
    size='cantidad',
    color='categoria',
    hover_name='subcategoria',
    title=' Ventas vs Margen por Subcategoría',
    labels={'ventas': 'Ventas ($)', 'margen': 'Margen (%)', 'cantidad': 'Cantidad'}
)

fig_scatter.update_layout(height=600)
fig_scatter.show()

print(" Top 10 subcategorías por ventas:")
display(top_subcategorias.head(10)[['ventas', 'ganancia', 'margen']])

 Top 10 subcategorías por ventas:


ventas  ganancia  margen
categoria       subcategoria                             
Technology      Phones        330007.05  44515.73   13.49
Furniture       Chairs        328449.10  26590.17    8.10
Office Supplies Storage       223843.61  21278.83    9.51
Furniture       Tables        206965.53 -17725.48   -8.56
Office Supplies Binders       203412.73  30221.76   14.86
Technology      Machines      189238.63   3384.76    1.79
                Accessories   167380.32  41936.64   25.05
                Copiers       149528.03  55617.82   37.20
Furniture       Bookcases     114880.00  -3472.56   -3.02
Office Supplies Appliances    107532.16  18138.01   16.87

## 7. Análisis de Segmentación de Clientes

Analizamos el comportamiento y valor de diferentes segmentos de clientes.

In [9]:
# Calculamos métricas detalladas por cliente
metricas_cliente = calcular_metricas_cliente(df)

# Análisis por segmento
analisis_segmento = df.groupby('segmento').agg({
    'ventas': ['sum', 'mean', 'count'],
    'ganancia': ['sum', 'mean'],
    'cantidad': 'sum',
    'descuento': 'mean',
    'id_cliente': 'nunique'
}).round(2)

# Aplanamos columnas multinivel
analisis_segmento.columns = ['_'.join(col) for col in analisis_segmento.columns]
analisis_segmento['ticket_promedio'] = (analisis_segmento['ventas_sum'] / analisis_segmento['ventas_count']).round(2)
analisis_segmento['valor_cliente_promedio'] = (analisis_segmento['ventas_sum'] / analisis_segmento['id_cliente_nunique']).round(2)

# Gráficos de segmentación
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribución de Ventas por Segmento', 'Valor Promedio por Cliente', 
                   'Frecuencia de Compra por Segmento', 'Margen por Segmento'),
    specs=[[{'type': 'pie'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}]]
)

# Pie chart de ventas por segmento
fig.add_trace(
    go.Pie(labels=analisis_segmento.index, values=analisis_segmento['ventas_sum'],
           name="Ventas"),
    row=1, col=1
)

# Valor promedio por cliente
fig.add_trace(
    go.Bar(x=analisis_segmento.index, y=analisis_segmento['valor_cliente_promedio'],
           marker_color='lightblue', name='Valor Cliente'),
    row=1, col=2
)

# Frecuencia de compra
frecuencia_compra = analisis_segmento['ventas_count'] / analisis_segmento['id_cliente_nunique']
fig.add_trace(
    go.Bar(x=analisis_segmento.index, y=frecuencia_compra,
           marker_color='lightgreen', name='Freq. Compra'),
    row=2, col=1
)

# Margen por segmento
margen_segmento = (analisis_segmento['ganancia_sum'] / analisis_segmento['ventas_sum'] * 100)
fig.add_trace(
    go.Bar(x=analisis_segmento.index, y=margen_segmento,
           marker_color='salmon', name='Margen %'),
    row=2, col=2
)

fig.update_layout(
    title="👥 Análisis de Segmentación de Clientes",
    height=800,
    showlegend=False
)

fig.show()

print(" Resumen por segmento:")
display(analisis_segmento[['ventas_sum', 'ganancia_sum', 'id_cliente_nunique', 'valor_cliente_promedio', 'ticket_promedio']])

 Resumen por segmento:


,ventas_sum,ganancia_sum,id_cliente_nunique,valor_cliente_promedio,ticket_promedio
segmento,,,,,
Consumer,1161401.34,134119.21,409,2839.61,223.73
Corporate,706146.37,91979.13,236,2992.15,233.82
Home Office,429653.15,60298.68,148,2903.06,240.97


## 8. Análisis de Descuentos y Rentabilidad

Analizamos el impacto de los descuentos en las ventas y la rentabilidad.

In [11]:
# Creamos rangos de descuento
df['rango_descuento'] = pd.cut(df['descuento'], 
                              bins=[0, 0.1, 0.2, 0.3, 0.5, 1.0],
                              labels=['0-10%', '10-20%', '20-30%', '30-50%', '50%+'],
                              include_lowest=True)

# Análisis por rango de descuento
analisis_descuentos = df.groupby('rango_descuento').agg({
    'ventas': ['sum', 'mean', 'count'],
    'ganancia': ['sum', 'mean'],
    'cantidad': 'sum'
}).round(2)

analisis_descuentos.columns = ['_'.join(col) for col in analisis_descuentos.columns]
analisis_descuentos['margen'] = (analisis_descuentos['ganancia_sum'] / analisis_descuentos['ventas_sum'] * 100).round(2)

# Gráficos de análisis de descuentos
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Ventas por Rango de Descuento', 'Margen por Rango de Descuento',
                   'Relación Descuento vs Ganancia', 'Distribución de Descuentos'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'scatter'}, {'type': 'histogram'}]]
)

# Ventas por rango de descuento
fig.add_trace(
    go.Bar(x=analisis_descuentos.index.astype(str), y=analisis_descuentos['ventas_sum'],
           marker_color='skyblue', name='Ventas'),
    row=1, col=1
)

# Margen por rango de descuento
fig.add_trace(
    go.Bar(x=analisis_descuentos.index.astype(str), y=analisis_descuentos['margen'],
           marker_color='lightcoral', name='Margen'),
    row=1, col=2
)

# Scatter plot descuento vs ganancia
sample_data = df.sample(1000)  # Tomamos una muestra para mejor visualización
fig.add_trace(
    go.Scatter(x=sample_data['descuento'], y=sample_data['ganancia'],
               mode='markers', marker=dict(color='green'), opacity=0.6,
               name='Descuento vs Ganancia'),
    row=2, col=1
)

# Histograma de descuentos
fig.add_trace(
    go.Histogram(x=df['descuento'], nbinsx=30, marker_color='orange',
                name='Distribución'),
    row=2, col=2
)

fig.update_layout(
    title=" Análisis de Descuentos y Rentabilidad",
    height=800,
    showlegend=False
)

fig.show()

print(" Análisis por rango de descuento:")
display(analisis_descuentos[['ventas_sum', 'ganancia_sum', 'margen', 'ventas_count']])

 Análisis por rango de descuento:


,ventas_sum,ganancia_sum,margen,ventas_count
rango_descuento,,,,
0-10%,1142277.82,330016.78,28.89,4892
10-20%,792152.89,91756.30,11.58,3709
20-30%,103226.66,-10369.28,-10.05,227
30-50%,195314.76,-48447.73,-24.80,310
50%+,64228.74,-76559.05,-119.20,856


## 9. Análisis de Métodos de Envío

Evaluamos el rendimiento y preferencias por método de envío.

In [12]:
# Análisis por método de envío
analisis_envio = df.groupby('modo_envio').agg({
    'ventas': ['sum', 'mean', 'count'],
    'ganancia': ['sum', 'mean'],
    'dias_envio': 'mean',
    'cantidad': 'sum'
}).round(2)

analisis_envio.columns = ['_'.join(col) for col in analisis_envio.columns]
analisis_envio['margen'] = (analisis_envio['ganancia_sum'] / analisis_envio['ventas_sum'] * 100).round(2)

# Gráficos de métodos de envío
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Preferencia de Métodos de Envío', 'Ventas por Método de Envío',
                   'Tiempo Promedio de Envío', 'Margen por Método de Envío'),
    specs=[[{'type': 'pie'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}]]
)

# Pie chart de preferencias
fig.add_trace(
    go.Pie(labels=analisis_envio.index, values=analisis_envio['ventas_count'],
           name="Preferencias"),
    row=1, col=1
)

# Ventas por método
fig.add_trace(
    go.Bar(x=analisis_envio.index, y=analisis_envio['ventas_sum'],
           marker_color='steelblue', name='Ventas'),
    row=1, col=2
)

# Tiempo promedio de envío
fig.add_trace(
    go.Bar(x=analisis_envio.index, y=analisis_envio['dias_envio_mean'],
           marker_color='orange', name='Días Envío'),
    row=2, col=1
)

# Margen por método
fig.add_trace(
    go.Bar(x=analisis_envio.index, y=analisis_envio['margen'],
           marker_color='lightgreen', name='Margen'),
    row=2, col=2
)

fig.update_layout(
    title=" Análisis de Métodos de Envío",
    height=800,
    showlegend=False
)

fig.show()

print(" Resumen por método de envío:")
display(analisis_envio[['ventas_sum', 'ganancia_sum', 'margen', 'dias_envio_mean', 'ventas_count']])

 Resumen por método de envío:


,ventas_sum,ganancia_sum,margen,dias_envio_mean,ventas_count
modo_envio,,,,,
First Class,351428.42,48969.84,13.93,2.18,1538
Same Day,128363.12,15891.76,12.38,0.04,543
Second Class,459193.57,57446.64,12.51,3.24,1945
Standard Class,1358215.74,164088.79,12.08,5.01,5968


## 10. Dashboard Ejecutivo Interactivo

Creamos un dashboard ejecutivo con las métricas más importantes.

In [15]:
# Preparamos datos para el dashboard ejecutivo
resumen_ejecutivo = {
    'ventas_totales': df['ventas'].sum(),
    'ganancia_total': df['ganancia'].sum(),
    'margen_promedio': (df['ganancia'].sum() / df['ventas'].sum()) * 100,
    'total_clientes': df['id_cliente'].nunique(),
    'total_pedidos': df['id_pedido'].nunique(),
    'ticket_promedio': df['ventas'].sum() / df['id_pedido'].nunique(),
    'productos_vendidos': df['cantidad'].sum(),
    'descuento_promedio': df['descuento'].mean() * 100
}

# Top performers
top_categoria = df.groupby('categoria')['ventas'].sum().sort_values(ascending=False).iloc[0]
top_categoria_nombre = df.groupby('categoria')['ventas'].sum().sort_values(ascending=False).index[0]
top_region = df.groupby('region')['ventas'].sum().sort_values(ascending=False).iloc[0]
top_region_nombre = df.groupby('region')['ventas'].sum().sort_values(ascending=False).index[0]

# Crear dashboard con múltiples métricas y mejor distribución visual
fig = make_subplots(
    rows=3, cols=4,
    subplot_titles=('Ventas Totales', 'Ganancia Total', 'Margen %', 'Total Clientes',
                   'Total Pedidos', 'Ticket Promedio', 'Productos Vendidos', 'Descuento Promedio',
                   'Top Categoría', 'Top Región', 'Tendencia Mensual', 'Distribución Segmentos'),
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}],
           [{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}],
           [{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'scatter'}, {'type': 'pie'}]],
    vertical_spacing=0.13,  # Más espacio vertical
    horizontal_spacing=0.10 # Más espacio horizontal
 )

# Indicadores principales (primeras 2 filas) con mayor tamaño de fuente y títulos compactos
indicadores = [
    (resumen_ejecutivo['ventas_totales'], "<b>Ventas Totales ($)</b>", ",.0f", 1, 1),
    (resumen_ejecutivo['ganancia_total'], "<b>Ganancia Total ($)</b>", ",.0f", 1, 2),
    (resumen_ejecutivo['margen_promedio'], "<b>Margen Promedio (%)</b>", ".1f", 1, 3),
    (resumen_ejecutivo['total_clientes'], "<b>Total Clientes</b>", ",d", 1, 4),
    (resumen_ejecutivo['total_pedidos'], "<b>Total Pedidos</b>", ",d", 2, 1),
    (resumen_ejecutivo['ticket_promedio'], "<b>Ticket Promedio ($)</b>", ",.0f", 2, 2),
    (resumen_ejecutivo['productos_vendidos'], "<b>Productos Vendidos</b>", ",d", 2, 3),
    (resumen_ejecutivo['descuento_promedio'], "<b>Descuento Promedio (%)</b>", ".1f", 2, 4),
    (top_categoria, f"<b>Top Categoría:</b> {top_categoria_nombre}", ",.0f", 3, 1),
    (top_region, f"<b>Top Región:</b> {top_region_nombre}", ",.0f", 3, 2)
 ]

for valor, titulo, formato, fila, col in indicadores:
    fig.add_trace(go.Indicator(
        mode="number",
        value=valor,
        title={"text": titulo, "font": {"size": 15}},
        number={'valueformat': formato, 'font': {'size': 30}}
    ), row=fila, col=col)

# Tendencia mensual
ventas_mes_simple = df.groupby(df['fecha_pedido'].dt.to_period('M'))['ventas'].sum()
fig.add_trace(
    go.Scatter(x=[str(x) for x in ventas_mes_simple.index], y=ventas_mes_simple.values,
               mode='lines+markers', name='Tendencia',
               line=dict(width=3, color='royalblue')),
    row=3, col=3
 )

# Distribución por segmentos
segmento_ventas = df.groupby('segmento')['ventas'].sum()
fig.add_trace(
    go.Pie(labels=segmento_ventas.index, values=segmento_ventas.values,
           name="Segmentos", textinfo='label+percent', insidetextorientation='radial'),
    row=3, col=4
 )

fig.update_layout(
    title="📊 DASHBOARD EJECUTIVO - SUPERSTORE",
    height=1100,  # Más alto para evitar solapamiento
    width=1400,   # Más ancho para mejor distribución
    font=dict(size=15),
    margin=dict(t=90, l=30, r=30, b=30),
    plot_bgcolor='white',
    paper_bgcolor='white',
    title_font_size=26,
    title_x=0.5,
    title_y=0.97,
    title_font_family='Arial',
    legend=dict(font=dict(size=13))
 )

# Forzar renderizado en Jupyter y mostrar advertencia si no se ve
import plotly.io as pio
pio.renderers.default = 'notebook_connected'
from IPython.display import display, HTML
display(HTML('<b>Si no ves el dashboard, prueba reiniciar el kernel o abrir en Jupyter Lab.</b>'))
fig.show()

## 11. Guardamos las visualizaciones

Exportamos las visualizaciones principales para reportes y presentaciones.

In [17]:
# Guardamos algunas visualizaciones clave
print(" Guardando visualizaciones principales...")

import os

# Crear carpeta de salida si no existe
os.makedirs('../outputs/graficos', exist_ok=True)

# Gráfico de evolución temporal de ventas (usando ventas_mes_simple)
import plotly.graph_objects as go

fig_temporal = go.Figure()
fig_temporal.add_trace(go.Scatter(
    x=[str(x) for x in ventas_mes_simple.index],
    y=ventas_mes_simple.values,
    mode='lines+markers',
    name='Ventas mensuales',
    line=dict(color='royalblue', width=3)
))
fig_temporal.update_layout(
    title="Evolución Temporal de Ventas",
    xaxis_title="Mes",
    yaxis_title="Ventas ($)",
    template="plotly_white"
)
fig_temporal.write_html('../outputs/graficos/evolucion_temporal.html')

# Gráfico de top productos por ventas (usando top_subcategorias)
top_prod = top_subcategorias.reset_index().copy()
fig_productos = go.Figure()
fig_productos.add_trace(go.Bar(
    x=top_prod['ventas'],
    y=top_prod['subcategoria'] + " (" + top_prod['categoria'] + ")",
    orientation='h',
    marker_color='teal'
))
fig_productos.update_layout(
    title="Top 15 Subcategorías por Ventas",
    xaxis_title="Ventas ($)",
    yaxis_title="Subcategoría",
    template="plotly_white",
    height=600
)
fig_productos.write_html('../outputs/graficos/top_productos.html')

print(" Visualizaciones guardadas en la carpeta outputs/graficos/")
print(" Archivos disponibles:")
print("  • evolucion_temporal.html")
print("  • top_productos.html")

 Guardando visualizaciones principales...
 Visualizaciones guardadas en la carpeta outputs/graficos/
 Archivos disponibles:
  • evolucion_temporal.html
  • top_productos.html


## 12. Resumen de Insights y Recomendaciones

Resumimos los principales hallazgos y recomendaciones del análisis visual.

In [18]:
print(" RESUMEN DE INSIGHTS Y RECOMENDACIONES")
print("=" * 60)

# Insights temporales
mes_mayor_venta = df.groupby('mes')['ventas'].sum().idxmax()
mes_menor_venta = df.groupby('mes')['ventas'].sum().idxmin()
print(f"\n PATRONES TEMPORALES:")
print(f"• Mes con mayores ventas: {mes_mayor_venta}")
print(f"• Mes con menores ventas: {mes_menor_venta}")
print(f"• Tiempo promedio de envío: {df['dias_envio'].mean():.1f} días")

# Insights geográficos
mejor_region = df.groupby('region')['ganancia'].sum().idxmax()
peor_margen_region = df.groupby('region').apply(lambda x: (x['ganancia'].sum() / x['ventas'].sum() * 100)).idxmin()
print(f"\n INSIGHTS GEOGRÁFICOS:")
print(f"• Región más rentable: {mejor_region}")
print(f"• Región con menor margen: {peor_margen_region}")

# Insights de productos
categoria_mas_rentable = df.groupby('categoria').apply(lambda x: (x['ganancia'].sum() / x['ventas'].sum() * 100)).idxmax()
productos_perdida = (df['ganancia'] < 0).sum()
print(f"\n INSIGHTS DE PRODUCTOS:")
print(f"• Categoría más rentable: {categoria_mas_rentable}")
print(f"• Productos con pérdidas: {productos_perdida} ({(productos_perdida/len(df)*100):.1f}%)")

# Insights de clientes
segmento_mayor_valor = df.groupby('segmento')['ventas'].sum().idxmax()
segmento_mayor_margen = df.groupby('segmento').apply(lambda x: (x['ganancia'].sum() / x['ventas'].sum() * 100)).idxmax()
print(f"\n INSIGHTS DE CLIENTES:")
print(f"• Segmento con mayores ventas: {segmento_mayor_valor}")
print(f"• Segmento con mayor margen: {segmento_mayor_margen}")

# Recomendaciones estratégicas
print(f"\n RECOMENDACIONES ESTRATÉGICAS:")
print(f"• Optimizar descuentos: Alta correlación negativa entre descuento y ganancia")
print(f"• Enfocar en productos rentables de la categoría {categoria_mas_rentable}")
print(f"• Desarrollar estrategias específicas para la región {mejor_region}")
print(f"• Revisar productos con pérdidas consistentes")
print(f"• Optimizar tiempos de envío en métodos más lentos")

print(f"\n Análisis de visualizaciones completado")
print(f" Dashboard ejecutivo y gráficos listos para presentación")

 RESUMEN DE INSIGHTS Y RECOMENDACIONES

 PATRONES TEMPORALES:
• Mes con mayores ventas: 11
• Mes con menores ventas: 2
• Tiempo promedio de envío: 4.0 días

 INSIGHTS GEOGRÁFICOS:
• Región más rentable: West
• Región con menor margen: Central

 INSIGHTS DE PRODUCTOS:
• Categoría más rentable: Technology
• Productos con pérdidas: 1871 (18.7%)

 INSIGHTS DE CLIENTES:
• Segmento con mayores ventas: Consumer
• Segmento con mayor margen: Home Office

 RECOMENDACIONES ESTRATÉGICAS:
• Optimizar descuentos: Alta correlación negativa entre descuento y ganancia
• Enfocar en productos rentables de la categoría Technology
• Desarrollar estrategias específicas para la región West
• Revisar productos con pérdidas consistentes
• Optimizar tiempos de envío en métodos más lentos

 Análisis de visualizaciones completado
 Dashboard ejecutivo y gráficos listos para presentación


## Conclusiones visuales

A través de las visualizaciones:

- Observamos la evolución de las ventas y detectamos tendencias temporales.
- Identificamos las subcategorías más relevantes para el negocio.
- Analizamos el impacto de los descuentos en la rentabilidad.

Como equipo, proponemos:
- Profundizar en el análisis de segmentos y regiones con mayor potencial.
- Evaluar estrategias para optimizar descuentos y márgenes.
- Preparar los datos para modelado y predicción en el siguiente notebook.

Continuamos con el desarrollo de modelos para extraer aún más valor de los datos.